In [ ]:
import librosa
import numpy as np
import torch
import torchaudio
import matplotlib.pyplot as plt
import os

from birdclassification.preprocessing.utils import *

In [ ]:
input_filename = '../../tests/sounds/cygnus_olor_2.mp3'
sr = 32000
n_fft = 512 
hop_length = 3 * 128
sample_length = 3

In [ ]:
def plot_librosa_spectrogram(y):
  M_db = librosa.power_to_db(y, ref=np.max)
  spectrogram = (M_db + 80) / 80
  return array_to_image(spectrogram)

In [ ]:
def plot_torch_spectrogram(specgram, title=None, ylabel="freq_bin", ax=None):
    if ax is None:
        _, ax = plt.subplots(1, 1)
    if title is not None:
        ax.set_title(title)
    ax.set_ylabel(ylabel)
    ax.imshow(librosa.power_to_db(specgram), origin="lower", aspect="auto", interpolation="nearest")

In [ ]:
y_librosa, sr = librosa.load(input_filename, sr=sr)
y_librosa, sr

In [ ]:
y_torch, sr2 = torchaudio.load(input_filename)
y_torch, sr2

In [ ]:
if sr2 != sr:
  y_torch = torchaudio.transforms.Resample(orig_freq=sr2, new_freq=sr)(y_torch)

y_torch, sr

In [ ]:
@timer
def create_spectrogram_torch(y):
  transform = torchaudio.transforms.MelSpectrogram(sample_rate=sr, n_fft=n_fft, hop_length=hop_length, f_min=150, f_max=15000, n_mels=64)
  return transform(y)[0]

@timer
def create_spectrogram_librosa(y):
  return librosa.feature.melspectrogram(y=y, sr=sr, n_fft=n_fft, hop_length=hop_length, n_mels=64, fmin=150, fmax=15000)

In [ ]:
spectrogram_torch = create_spectrogram_torch(y_torch)

plot_torch_spectrogram(spectrogram_torch)

In [ ]:
spectrogram_librosa = create_spectrogram_librosa(y_librosa)

plot_torch_spectrogram(spectrogram_librosa)

In [ ]:
img_torch = plot_librosa_spectrogram(spectrogram_torch)
img_torch

In [ ]:
img_librosa = plot_librosa_spectrogram(spectrogram_librosa)
img_librosa

In [ ]:
y_librosa

In [ ]:
y_torch

In [ ]:
get_loudest_index(y_librosa, n_fft, hop_length)

In [ ]:
get_loudest_index(y_torch, n_fft, hop_length)

In [ ]:
cut_around_index_librosa(y_librosa, 1035520, sr * sample_length)

In [ ]:
cut_around_index(y_torch, 1035520, sr * sample_length)

In [ ]:
fragments_librosa = get_thresholded_fragments_librosa(y_librosa, sr, n_fft, hop_length, sample_length, 0.7)
spectrograms_librosa = [create_spectrogram_librosa(fragment) for fragment in fragments_librosa]
plot_torch_spectrogram(spectrograms_librosa[2])

In [ ]:
fragments_torch = get_thresholded_fragments(y_torch, sr, n_fft, hop_length, sample_length, 0.7)
spectrograms_torch = [create_spectrogram_torch(fragment) for fragment in fragments_torch]
plot_torch_spectrogram(spectrograms_torch[2])